In [14]:
import numpy as np
import os
import gzip

# Chemins des fichiers
DATA_DIR = "/Users/mariusavosse/Documents/M2_Paris_Cité/S2/DeepLearing/tp2/fashion"
TRAIN_IMAGES_FILE = os.path.join(DATA_DIR, "train-images-idx3-ubyte.gz")
TRAIN_LABELS_FILE = os.path.join(DATA_DIR, "train-labels-idx1-ubyte.gz")
TEST_IMAGES_FILE = os.path.join(DATA_DIR, "t10k-images-idx3-ubyte.gz")
TEST_LABELS_FILE = os.path.join(DATA_DIR, "t10k-labels-idx1-ubyte.gz")

TEXT_LABELS = ['t-shirt', 'trouser', 'pullover', 'dress', 'coat', 
               'sandal', 'shirt', 'sneaker', 'bag', 'ankle boot']

# Fonctions utilitaires pour charger les données
def load_images(file_path):
    with gzip.open(file_path, 'rb') as f:
        magic, num_images, rows, cols = np.frombuffer(f.read(16), dtype=np.uint32, count=4)
        num_images, rows, cols = num_images.byteswap(), rows.byteswap(), cols.byteswap()
        images = np.frombuffer(f.read(), dtype=np.uint8)
        return images.reshape(num_images, rows, cols) / 255.0

def load_labels(file_path):
    with gzip.open(file_path, 'rb') as f:
        magic, num_labels = np.frombuffer(f.read(8), dtype=np.uint32, count=2)
        num_labels = num_labels.byteswap()
        labels = np.frombuffer(f.read(), dtype=np.uint8)
        return labels

# Chargement des datasets
train_imgs = load_images(TRAIN_IMAGES_FILE)
train_labels = load_labels(TRAIN_LABELS_FILE)
test_imgs = load_images(TEST_IMAGES_FILE)
test_labels = load_labels(TEST_LABELS_FILE)

# Vérification
print(f"Train Images Shape: {train_imgs.shape}, Train Labels Shape: {train_labels.shape}")
print(f"Test Images Shape: {test_imgs.shape}, Test Labels Shape: {test_labels.shape}")


Train Images Shape: (60000, 28, 28), Train Labels Shape: (60000,)
Test Images Shape: (10000, 28, 28), Test Labels Shape: (10000,)


In [15]:
## Prétraitement des données
### Aplatir les images
train_imgs_flat = train_imgs.reshape(train_imgs.shape[0], -1)  # (60000, 784)
test_imgs_flat = test_imgs.reshape(test_imgs.shape[0], -1)    # (10000, 784)

print(f"Train Images Flat Shape: {train_imgs_flat.shape}")
print(f"Test Images Flat Shape: {test_imgs_flat.shape}")


Train Images Flat Shape: (60000, 784)
Test Images Flat Shape: (10000, 784)


In [16]:
## Implémentation de Softmax
def softmax(logits):
    max_logits = np.max(logits, axis=1, keepdims=True)
    exp_logits = np.exp(logits - max_logits)
    return exp_logits / np.sum(exp_logits, axis=1, keepdims=True)


In [17]:
## Perte d'Entropie Croisée
def cross_entropy_loss(probabilities, labels):
    return -np.mean(np.log(probabilities[np.arange(len(labels)), labels]))


In [18]:
## Calcul des Gradients
def compute_gradients(x, probabilities, labels):
    probabilities[np.arange(len(labels)), labels] -= 1
    grad_W = x.T @ probabilities / len(labels)
    grad_b = np.sum(probabilities, axis=0) / len(labels)
    return grad_W, grad_b


In [19]:
## Entraînement sur un Batch Unique

# Initialisation
np.random.seed(42)
W = np.random.normal(0, 0.01, (784, 10))
b = np.zeros(10)

learning_rate = 0.1
batch_size = 64
x_batch = train_imgs_flat[:batch_size]
y_batch = train_labels[:batch_size]

# Entraînement
for epoch in range(500):
    logits = x_batch @ W + b
    probabilities = softmax(logits)
    loss = cross_entropy_loss(probabilities, y_batch)
    grad_W, grad_b = compute_gradients(x_batch, probabilities, y_batch)
    W -= learning_rate * grad_W
    b -= learning_rate * grad_b
    if epoch % 50 == 0:
        print(f"Epoch {epoch}, Loss: {loss:.4f}")

# Vérification de la perte
print(f"Final Loss: {loss:.4f}")

Epoch 0, Loss: 2.2885
Epoch 50, Loss: 0.4300
Epoch 100, Loss: 0.2463
Epoch 150, Loss: 0.1690
Epoch 200, Loss: 0.1267
Epoch 250, Loss: 0.1004
Epoch 300, Loss: 0.0827
Epoch 350, Loss: 0.0701
Epoch 400, Loss: 0.0607
Epoch 450, Loss: 0.0534
Final Loss: 0.0478


In [20]:
## Entraînement Complet avec Mini-Batchs
def train_model(train_data, train_labels, epochs=10, batch_size=64, lr=0.1):
    global W, b
    for epoch in range(epochs):
        # Mélanger les données
        indices = np.arange(len(train_data))
        np.random.shuffle(indices)
        train_data = train_data[indices]
        train_labels = train_labels[indices]

        # Parcourir les mini-batchs
        for i in range(0, len(train_data), batch_size):
            x_batch = train_data[i:i+batch_size]
            y_batch = train_labels[i:i+batch_size]
            logits = x_batch @ W + b
            probabilities = softmax(logits)
            grad_W, grad_b = compute_gradients(x_batch, probabilities, y_batch)
            W -= lr * grad_W
            b -= lr * grad_b

        # Calculer et afficher la perte après chaque epoch
        logits = train_data @ W + b
        probabilities = softmax(logits)
        loss = cross_entropy_loss(probabilities, train_labels)
        print(f"Epoch {epoch+1}, Loss: {loss:.4f}")

# Entraînement
train_model(train_imgs_flat, train_labels)


Epoch 1, Loss: 0.4903
Epoch 2, Loss: 0.4924
Epoch 3, Loss: 0.4403
Epoch 4, Loss: 0.4441
Epoch 5, Loss: 0.4192
Epoch 6, Loss: 0.4302
Epoch 7, Loss: 0.4269
Epoch 8, Loss: 0.4133
Epoch 9, Loss: 0.4193
Epoch 10, Loss: 0.4410


In [21]:
## Evaluation
def evaluate_model(data, labels):
    logits = data @ W + b
    probabilities = softmax(logits)
    predictions = np.argmax(probabilities, axis=1)
    accuracy = np.mean(predictions == labels)
    return accuracy

# Évaluation
train_accuracy = evaluate_model(train_imgs_flat, train_labels)
test_accuracy = evaluate_model(test_imgs_flat, test_labels)
print(f"Train Accuracy: {train_accuracy:.2%}")
print(f"Test Accuracy: {test_accuracy:.2%}")


Train Accuracy: 84.90%
Test Accuracy: 83.27%
